# Exercise 6. ICT Project: Communication Services and Security
### Cèsar Fernàndez Camón

#### Authors:
- Albert Pérez Datsira
- Jeongyun Lee


## Problem 1
Let's consider a WEP (Wireless Encryption Protocol) cipher consisting on; 8 bits key length, 8 bits IV length, 8 bits CRC (Cyclic Redundancy Check) length being (x^8 + 1) the CRC polynomial.

In addition, the PRNG (Pseudorandom Number Generator) is implemented as 8 bits shift register,

denoting:
* PS(i) as the shift register status at iteration i
* PS(i)[j] as the j th bit of the shift register at iteration i. Consider P S(i)[0] as the most left bit
* PO(i) as the output bit of the PNRG at iteration i

being:
- PS(0) = IV ⊕ key
- PS(i)[0] = PS(i - 1)[4] ⊕ PS(i - 1)[7]
- PS(i)[j] = PS(i - 1)[j - 1], j > 0
- PO(i) = PS(i)[4] ⊕ PS(i)[7]

where (⊕) means a XOR operation.

1. Probe that the ciphered data for the clear data 0x0123, key=0x33 and IV=0x11 is: **0x667E92**
2. Probe how deciphering with a wrong key (key=0x22) an error condition is reported

## Utils
First of all, will be introduced the fucntions amount provided used throughout the code, to compute the solution.

First of all, will be introduced the modules and functions provided used throughout the code, to compute the solution.

We are using `prettytable` to print more structured the results and you may install it by executing

`$ pip install prettytable`

or using any other package manager such as `Anaconda` by `$conda install -c conda-forge prettytable`

In [1]:
from prettytable import PrettyTable

### Calculate CRC (Cyclic Redundancy Check)
Focused on generating the CRC, bits for error detection, for an inputted message and a divisor that is part of the WLAN Frame: MAC frame.

In [2]:
def crc(msg, div, code='00000000'):
    """Cyclic Redundancy Check
    Generates an error detecting code based on an inputted message
    and divisor in the form of a polynomial representation.
    Arguments:
        msg: The input message of which to generate the output code.
        div: The divisor in polynomial form. For example, if the polynomial
            of x^3 + x + 1 is given, this should be represented as '1011' in
            the div argument.
        code: This is an option argument where a previously generated code may
            be passed in. This can be used to check validity. If the inputted
            code produces an outputted code of all zeros, then the message has
            no errors.
    Returns:
        An error-detecting code generated by the message and the given divisor.
    """
    # Append the code to the message. If no code is given, default to '000'
    msg = msg + code

    # Convert msg and div into list form for easier handling
    msg = list(msg)
    div = list(div)

    # Loop over every message bit (minus the appended code)
    for i in range(len(msg)-len(code)):
        # If that messsage bit is 1, perform modulo 2 multiplication
        if msg[i] == '1':
            for j in range(len(div)):
                # Perform modulo 2 multiplication on each index of the divisor
                msg[i+j] = str((int(msg[i+j])+int(div[j]))%2)

    # Output the last error-checking code portion of the message generated
    return ''.join(msg[-len(code):])

### Print as binary

In [3]:
def hexToBin(hex, n=24):
  return f'{hex:0>{n}b}'

### PRNG (Pseudorandom Number Generator)
Next, stands the function regarding the PRNG, which is a WEP cypher algorithm, following the statement problem constraints and using the notations presented.

>Note: The PNRG is implemented as a 8 bits shift register

In [4]:
def xor(a, b): ## simply performing XOR operation among two input values
    return a ^ b

In [5]:
## from the initialization vector and the key there are PS & PO both calculated
def prng(IV, key):
  PS = ['{0:08b}'.format(xor(key, IV))]
  PO = []
  
  # iterating over PS once for each bit in ICV + Data
  for i in range(1, len(ICVData)):
    tmp = str(xor(int(PS[i-1][4]), int(PS[i-1][7])))
    PS.append(tmp)
    PO.append(tmp)

    for j in range(1, 8): # getting the other bits
      PS[i] += PS[i - 1][j - 1]

  # last PO value
  PO.append(PS[-1][0])
  
  return PS, PO;

### Data cyphering
- XOR (⊕) between PO and the Data + ICV

In [6]:
def cypher(PO, ICVData):
  cyphered = ''
  for i in range(0, len(ICVData)):
    cyphered += (str(int(PO[i]) ^ int(ICVData[i]))) # applying XOR among the data, WHERE ICVDATA = Data + ICV

  return cyphered

### Data decyphering
- XOR (⊕) between PO and the cyphered data received.

In [7]:
def decypher(PO, cyphered):
  decyphered = ''
  for i in range(0, len(cyphered)):
    decyphered += (str(int(PO[i]) ^ int(cyphered[i]))) # applying XOR
  return decyphered

### Matching results
Compares data values returning a boolean

In [8]:
def match(data1, data2):
  return data1 == data2

In [9]:
def bold(str):
    return "\033[1m{0}\033[0m".format(str)

In [10]:
def splitIn4(str):
    itr = map(''.join, zip(*[iter(str)]*4))
    str = ''
    for x in itr:
        str += x + " "
    return str[0:len(str)-1]

## Problem execution

### Input data
Defining the variables with the statement data, and then printing formatted on bits results.

In [11]:
data = 0x0123 # 16 bits
key = 0x33 # 8 bits
IV = 0x11 # 8 bits
divCRC = '100000001' # (x^8 + 1) CRC polynomial divisor ## 8 bits
expected = 0x667E92

In [12]:
inputData = PrettyTable(title="Input Data", field_names=["Field", "Code", "Binary"])

inputData.align["Field"] = "l"
inputData.align["Code"] = "r"
inputData.align["Binary"] = "r"

inputData.add_row([bold("Data"), hex(data), splitIn4(hexToBin(data, 16))])
inputData.add_row([bold("Key"), hex(key), splitIn4(hexToBin(key, 8))])
inputData.add_row([bold("IV"), hex(IV), splitIn4(hexToBin(IV, 8))])
inputData.add_row([bold("CRC divisor"), "(x^8 + 1)", splitIn4(divCRC)])
inputData.add_row([bold("Expected"),  hex(expected), splitIn4(hexToBin(expected))])                  

print(inputData)

+---------------------------------------------------------+
|                        Input Data                       |
+-------------+-----------+-------------------------------+
| Field       |      Code |                        Binary |
+-------------+-----------+-------------------------------+
| Data        |     0x123 |           0000 0001 0010 0011 |
| Key         |      0x33 |                     0011 0011 |
| IV          |      0x11 |                     0001 0001 |
| CRC divisor | (x^8 + 1) |                     1000 0000 |
| Expected    |  0x667e92 | 0110 0110 0111 1110 1001 0010 |
+-------------+-----------+-------------------------------+


## Problem 1.1
Probe that the ciphered data for the clear text data 0x0123, key=0x33 and IV=0x11 is: **0x667E92**

First, we know the cypher method to follow stands as:

<div style="width: 580px; margin-top: 20px;"><img src='assets/wep_cypher_method.png' alt="WEP Cyphert Method"/></div>

So, we must first get the ICV (Integrity Check Value) value from the message (data) and the CRC divisor.

Then, allow us to get the resultant ICV + Data value.

In [13]:
msg = hexToBin(data, 16) # 16 bits data (length frame)

ICV = crc(msg, divCRC) # getting the ICV using the crc() function. Being (x^8 + 1) the CRC polynomial.

ICVData = msg + ICV # ICV + Data stream generation, which will be cyphered afterward

Now, we must compute the PO value from the 16 bits data and 8 bits key, which is the result of the PRNG operation using it to apply the XOR operation.

In [14]:
PS, PO = prng(IV, key) # using the pnrg() function to generate PS and PO
    
# intermediate results
table = PrettyTable(field_names=["Field", "Value"])

table.align["Field"] = "l"
table.align["Value"] = "r"

table.add_row(["Msg", splitIn4(msg)])
table.add_row(["ICV", splitIn4(ICV)])
table.add_row(["ICVData", splitIn4(ICVData)])
table.add_row(["PO", splitIn4(''.join([str(el) for el in PO]))])
print(table)

+---------+-------------------------------+
| Field   |                         Value |
+---------+-------------------------------+
| Msg     |           0000 0001 0010 0011 |
| ICV     |                     0010 0010 |
| ICVData | 0000 0001 0010 0011 0010 0010 |
| PO      | 0110 0111 0101 1101 1011 0000 |
+---------+-------------------------------+


Then, the remaining step to cypher the input data is to make the XOR operation between the resultant PO and the ICV + Data

In [15]:
cyphered = cypher(PO, ICVData) # cyphering - XOR operation (⊕) between ICV + DATA and the PO 

tableCypher = PrettyTable(title="Cyphered data", field_names=["Field", "Code (hex)", "Binary"])
tableCypher.align["Field"] = "l"
tableCypher.align["Code (hex)"] = "r"
tableCypher.align["Binary"] = "r"

tableCypher.add_row([bold("Chyphered"), hex(int(cyphered, 2)), splitIn4(cyphered)])
tableCypher.add_row([bold("Expected"), hex(expected), splitIn4(hexToBin(expected))])
print(tableCypher)

# comparing if values are the same
print("{0} => {1}".format(bold(" Matching??"), str(match(cyphered, hexToBin(expected)))))

+--------------------------------------------------------+
|                     Cyphered data                      |
+-----------+------------+-------------------------------+
| Field     | Code (hex) |                        Binary |
+-----------+------------+-------------------------------+
| Chyphered |   0x667e92 | 0110 0110 0111 1110 1001 0010 |
| Expected  |   0x667e92 | 0110 0110 0111 1110 1001 0010 |
+-----------+------------+-------------------------------+
 Matching?? => True


As you may see above, the cyphered data its exactly the same as expected, but far away, once the data is decyphered we obtain the original ICV + Data back.

In [16]:
# in this case we can use the same PO value since we apply the same key and IV values
decyphered = decypher(PO, cyphered) # decyphering - XOR operation (⊕) between PO and the cyphered data.

tableDecypher = PrettyTable(title="Decyphered data", field_names=["Field", "Code (hex)", "Binary"])
tableDecypher.align["Field"] = "l"
tableDecypher.align["Code (hex)"] = "r"
tableDecypher.align["Binary"] = "r"

tableDecypher.add_row([bold("Decyphered"), hex(int(decyphered[:16],2)), bold(splitIn4(decyphered))])
tableDecypher.add_row([bold("ICVData"), hex(int(ICVData[:16],2)), bold(splitIn4(ICVData))])
tableDecypher.add_row([bold("Original"), hex(data), hexToBin(data, 16)])
print(tableDecypher)

# comparing if the values are the same
print("{0} => {1}".format(bold(" Matching??"), str(match(decyphered, ICVData))))

+---------------------------------------------------------+
|                     Decyphered data                     |
+------------+------------+-------------------------------+
| Field      | Code (hex) |                        Binary |
+------------+------------+-------------------------------+
| Decyphered |      0x123 | 0000 0001 0010 0011 0010 0010 |
| ICVData    |      0x123 | 0000 0001 0010 0011 0010 0010 |
| Original   |      0x123 |              0000000100100011 |
+------------+------------+-------------------------------+
 Matching?? => True


## Problem 1.2

Probe how deciphering with a wrong key (key=0x22) an error condition is reported.

First, the following is the decypher method

<div style="width: 580px; margin-top: 20px;"><img src='assets/wep_decypher_method.png' alt="WEP Decypher Method"/></div>

In [17]:
key = 0x22; # 8 bits
print("{0} {1} = {2}".format(bold("Key:"), hex(key), splitIn4(hexToBin(key, 8))))

Key: 0x22 = 0010 0010


Using the same cyphered value, but a different key to decypher the message. For that reason, we expect a new PO value to be obtained (PO2).

In [18]:
PS2, PO2 = prng(IV, key) # PO using the new key
print(splitIn4(PO2))

1010 1001 1110 0110 1101 0000


In [19]:
# decyphering with the wrong key
decyphered2 = decypher(PO2, cyphered) # XOR operation between the new PO and the same cyphered value

tableDecypher2 = PrettyTable(title="Decyphered data", field_names=["Field", "Code (hex)", "Binary"])
tableDecypher2.align["Field"] = "l"
tableDecypher2.align["Code (hex)"] = "r"
tableDecypher2.align["Binary"] = "r"

tableDecypher2.add_row([bold("Decyphered"), hex(int(decyphered2[:16],2)), bold(splitIn4(decyphered2))])
tableDecypher2.add_row([bold("ICVData"), hex(int(ICVData[:16],2)), bold(splitIn4(ICVData))])
tableDecypher2.add_row([bold("Original"), hex(data), splitIn4(hexToBin(data, 16))])
print(tableDecypher2)

# comparing if values are the same
print("{0} => {1}".format(bold(" Matching??"), str(match(decyphered2, ICVData))))

+---------------------------------------------------------+
|                     Decyphered data                     |
+------------+------------+-------------------------------+
| Field      | Code (hex) |                        Binary |
+------------+------------+-------------------------------+
| Decyphered |     0xcf98 | 1100 1111 1001 1000 0100 0010 |
| ICVData    |      0x123 | 0000 0001 0010 0011 0010 0010 |
| Original   |      0x123 |           0000 0001 0010 0011 |
+------------+------------+-------------------------------+
 Matching?? => False


Finally, the resultant decyphered data is different from the original ICV + Data, as expected when using a wrong key.